In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('csv'):
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-credit-risk-model-stability/sample_submission.csv
/kaggle/input/home-credit-credit-risk-model-stability/feature_definitions.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_6.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_11.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_0.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_9.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_base.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_1_1.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_b_2.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_10.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_cre

# Data Cleaning


In [2]:
# drop columns with more than 90% null data
def drop_cols(df):
    for i in df.columns:
        null_values = df[f'{i}'].is_null().mean()
    if null_values > 0.9:
        df=df.drop(f'{i}')
    return  df
 
# convert data types 
def data_types(df):
    for i in df.columns:
        if i[-1] in ('D'):
            df = df.with_columns(pl.col(i).cast(pl.Date))
        if i[-1] in ('M'):
            df= df.with_columns(pl.col(i).cast(pl.String))
        if i[-1] in ('A'):
            df= df.with_columns(pl.col(i).cast(pl.Float64))
        if i[-1] in ('P'):
            df= df.with_columns(pl.col(i).cast(pl.Int16))
    return df
        

# train_applprev
train_applprev_1_0,train_applprev_1_1,train_applprev_2 are internal files with depth 1,1,2 respectively.

In [ ]:
tr_applprev_0=pl.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_1_0.csv')
tr_applprev_1=pl.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_1_1.csv')
tr_applprev_2=pl.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_2.csv')

Join works differently in pandas and polars.In pandas the joining happens on index-index or index-column and merge happens on column(s) given by us. But in polars there is no merge option but only join and "on" columns are consider from both dataframes.

In [ ]:
tr_applprev = pl.concat([tr_applprev_0,tr_applprev_1])
tr_applprev=tr_applprev.join(tr_applprev_2,how='left',on=['case_id','num_group1'])

In [ ]:
tr_applprev=drop_cols(tr_applprev)
tr_applprev=data_types(tr_applprev)
tr_applprev.describe()

In [ ]:
print([i for i in tr_applprev.columns if i[-1] in ('D')])


In [ ]:
# creating new columns from date columns
tr_applprev['contract_duration']= tr_applprev['dtlastpmtallstes_3545839D'].dt.days()- tr_applprev['dateactivated_425D'].dt.days()
# tr_applprev['differ_days_of payment'] = tr_applprev['dtlastpmt_581D'] - tr_applprev['dtlastpmtallstes_3545839D']
# tr_applprev['no_of_employement_days'] =  tr_applprev['approvaldate_319D'] - tr_applprev['employedfrom_700D']
# tr_applprev.columns

Aggregation of the data-
* actualdpd_943P - max
* annuity_853A - mean
* days_to_approval - approvaldate_319D-creationdate_885D 
* byoccupationinc_3656910L -  mean
* cancelreason_3545846M -###
* childnum_21L - max
* credacc_actualbalance_314A - mean
* credacc_credlmt_575A - mean
* credacc_maxhisbal_375A - mean
* credacc_minhisbal_90A  - mean
* credacc_status_367L - ###
* credacc_transactions_402L - mean
* credamount_590A - mean
* credtype_587L - ###
* currdebt_94A - max
* dateactivated_425D - ###
* district_544M - ###
* downpmt_134A - mean
* dtlastpmt_581D- ### loan tenure can be calculated
* dtlastpmtallstes_3545839D - ###
* education_1138M - max
* employedfrom_700D - ###
* familystate_726L- ##
* firstnonzeroinstldate_307D - ##
* inittransactioncode_279L - ##
* isbidproduct_390L - ## company offering other services (true)
* isdebitcard_527L - product of other service is debit card or not
* mainoccupationinc_437A - max
* maxdpdtolerance_577P - max
* outstandingdebt_522A - max
* pmtnum_8L- max
* postype_4733339M - ##
* profession_152M - ##
* rejectreason_755M- ##
* rejectreasonclient_4145042M - ##
* revolvingaccount_394A - mean
* status_219L -##
* tenor_203L - mean
* cacccardblochreas_147M - ## pivot
* conts_type_509L - ## pivot
* credacc_cards_status_52L - ## pivot


In [ ]:
tr_applprev.head()

In [ ]:
tr_applprev['credacc_cards_status_52L'].unique()

In [ ]:
result=tr_applprev.group_by('case_id').agg(pl.col('cancelreason_3545846M').count().sort(descending=True))


In [ ]:
tr_applprev.filter(tr_applprev['case_id']==3)

In [ ]:
pl.Config(fmt_str_lengths=1000)
pl.Config.set_tbl_rows(500)
feature_definition= pl.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/feature_definitions.csv')

feature_definition.head(500)#[feature_definition.Variable=='persontype_792L']
